In [3]:
import requests
import os
import pandas as pd
import random
from datetime import datetime, timedelta
from uuid import uuid4

# CONFIGURA TU API KEY AQUÍ
API_KEY = '51425029-473b8fc89f2a01cd210f9fc88'

# Configuración de directorios
BASE_DIR = '/Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DIGITAL_Proyecto Final'
IMAGES_DIR = os.path.join(BASE_DIR, 'imagenes_descargadas')
os.makedirs(BASE_DIR, exist_ok=True)
os.makedirs(IMAGES_DIR, exist_ok=True)

# Temáticas de búsqueda
tematicas = ['comida', 'viajes', 'moda', 'tecnología', 'mascotas', 'deporte', 'salud', 'educación', 'arte', 'naturaleza']

# Parámetros de búsqueda
NUM_IMAGENES = 200
IMAGENES_POR_TEMATICA = NUM_IMAGENES // len(tematicas)

# Configuración de fechas: del 1 de agosto de 2024 al 1 de agosto de 2025
fecha_inicio = datetime(2024, 8, 1)
fecha_fin = datetime(2025, 8, 1)
total_dias = (fecha_fin - fecha_inicio).days

# Generar fechas distribuidas aproximadamente 3 veces por semana
def generar_fechas_distribuidas(num_fechas, fecha_inicio, total_dias):
    fechas = []
    # Aproximadamente 3 publicaciones por semana = intervalo promedio de 2-3 días
    intervalo_base = 2  # días base entre publicaciones
    
    fecha_actual = fecha_inicio
    for i in range(num_fechas):
        # Agregar variación aleatoria de 0-2 días al intervalo base
        dias_añadir = intervalo_base + random.randint(0, 2)
        fecha_actual += timedelta(days=dias_añadir)
        
        # Asegurar que no excedemos la fecha límite
        if fecha_actual > fecha_fin:
            # Si excedemos, usar una fecha aleatoria en el rango restante
            dias_restantes = (fecha_fin - fecha_inicio).days
            fecha_actual = fecha_inicio + timedelta(days=random.randint(0, dias_restantes))
        
        fechas.append(fecha_actual)
    
    # Mezclar las fechas para que no estén en orden secuencial
    random.shuffle(fechas)
    return fechas

# Generar todas las fechas de una vez
fechas_publicacion = generar_fechas_distribuidas(NUM_IMAGENES, fecha_inicio, total_dias)
indice_fecha = 0

# Contador global para IDs simplificados
contador_imagen = 1

datos = []

for tema in tematicas:
    print(f"Descargando imágenes de temática: {tema}")
    page = 1
    descargadas = 0

    while descargadas < IMAGENES_POR_TEMATICA:
        url = f"https://pixabay.com/api/?key={API_KEY}&q={tema}&image_type=photo&per_page=20&page={page}"
        response = requests.get(url)
        data = response.json()

        if 'hits' not in data or not data['hits']:
            break

        for hit in data['hits']:
            image_url = hit['webformatURL']
            
            # ID simplificado: POST_1, POST_2, POST_3, etc.
            post_id = f"POST_{contador_imagen}"
            contador_imagen += 1
            
            # Usar fecha de la lista pre-generada
            post_date = fechas_publicacion[indice_fecha % len(fechas_publicacion)]
            indice_fecha += 1
            
            # Usar el mismo ID para el nombre del archivo
            filename = f"{post_id}.jpg"
            image_path = os.path.join(IMAGES_DIR, filename)

            try:
                img_data = requests.get(image_url).content
                with open(image_path, 'wb') as handler:
                    handler.write(img_data)
                
                datos.append({
                    'id_post': post_id,
                    'Fecha': post_date.strftime('%Y-%m-%d'),
                    'Imagen': filename,
                    'Ruta': image_path
                })

                descargadas += 1
                if descargadas >= IMAGENES_POR_TEMATICA:
                    break
            except Exception as e:
                print(f"Error descargando imagen: {e}")

        page += 1

# Guardar CSV en la carpeta del proyecto
csv_path = os.path.join(BASE_DIR, 'publicaciones_pixabay.csv')
df = pd.DataFrame(datos)
# Ordenar por fecha para mejor visualización
df = df.sort_values('Fecha')
df.to_csv(csv_path, index=False)

print("✅ Descarga completa. CSV generado en el proyecto")
print(f"📁 Directorio del proyecto: {BASE_DIR}")
print(f"📄 CSV guardado en: {csv_path}")
print(f"📅 Fechas desde {fecha_inicio.strftime('%d/%m/%Y')} hasta {fecha_fin.strftime('%d/%m/%Y')}")
print(f"📊 Total de publicaciones: {len(datos)}")

# Mostrar estadísticas de distribución
df['Fecha'] = pd.to_datetime(df['Fecha'])
publicaciones_por_mes = df.groupby(df['Fecha'].dt.to_period('M')).size()
print("\n📈 Distribución por mes:")
for mes, cantidad in publicaciones_por_mes.items():
    print(f"  {mes}: {cantidad} publicaciones")

# Mostrar muestra de los primeros registros
print("\n📋 Muestra de los primeros 5 registros:")
print(df.head())

Descargando imágenes de temática: comida
Descargando imágenes de temática: viajes
Descargando imágenes de temática: moda
Descargando imágenes de temática: tecnología
Descargando imágenes de temática: mascotas
Descargando imágenes de temática: deporte
Descargando imágenes de temática: salud
Descargando imágenes de temática: educación
Descargando imágenes de temática: arte
Descargando imágenes de temática: naturaleza
✅ Descarga completa. CSV generado en el proyecto
📁 Directorio del proyecto: /Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DIGITAL_Proyecto Final
📄 CSV guardado en: /Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DIGITAL_Proyecto Final/publicaciones_pixabay.csv
📅 Fechas desde 01/08/2024 hasta 01/08/2025
📊 Total de publicaciones: 200

📈 Distribución por mes:
  2024-08: 10 publicaciones
  2024-09: 10 publicaciones
  2024-10: 10 publicaciones
  2024-11: 10 publicaciones
  2024-12: 12 publicaciones
  2025-01: 12 publicaciones
  2025-02: 12 publicaciones
  2025-03: 31 publicaciones
  20

In [4]:
import zipfile
import os

# Configuración de directorios
BASE_DIR = '/Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DIGITAL_Proyecto Final'
image_folder = os.path.join(BASE_DIR, 'imagenes_descargadas')
zip_filename = os.path.join(BASE_DIR, 'imagenes_pixabay.zip')

# Crear el archivo .zip
with zipfile.ZipFile(zip_filename, 'w') as zipf:
    for filename in os.listdir(image_folder):
        if filename.endswith('.jpg'):
            filepath = os.path.join(image_folder, filename)
            zipf.write(filepath, arcname=filename)

print(f"✅ Archivo .zip creado: {zip_filename}")

✅ Archivo .zip creado: /Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DIGITAL_Proyecto Final/imagenes_pixabay.zip


In [7]:
# Verificar archivos creados y mostrar estadísticas finales
import os
import pandas as pd

# Configuración de directorios
BASE_DIR = '/Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DIGITAL_Proyecto Final'
IMAGES_DIR = os.path.join(BASE_DIR, 'imagenes_descargadas')

print(f"📁 Directorio del proyecto: {BASE_DIR}")
print("\n📁 Archivos generados:")
archivos_importantes = [f for f in os.listdir(BASE_DIR) if f.endswith(('.csv', '.zip'))]
for archivo in archivos_importantes:
    archivo_path = os.path.join(BASE_DIR, archivo)
    if os.path.exists(archivo_path):
        tamaño = os.path.getsize(archivo_path) / (1024 * 1024)  # Tamaño en MB
        print(f"  ✅ {archivo} - {tamaño:.2f} MB")

print(f"\n📂 Carpeta de imágenes: {IMAGES_DIR}")
if os.path.exists(IMAGES_DIR):
    num_imagenes = len([f for f in os.listdir(IMAGES_DIR) if f.endswith('.jpg')])
    print(f"  📸 Imágenes descargadas: {num_imagenes}")
    
    # Mostrar algunas imágenes de ejemplo
    ejemplos = sorted([f for f in os.listdir(IMAGES_DIR) if f.endswith('.jpg')])[:10]
    print(f"  📋 Ejemplos de nombres de archivo: {', '.join(ejemplos)}")

# Verificar el CSV generado
csv_path = os.path.join(BASE_DIR, 'publicaciones_pixabay.csv')
if os.path.exists(csv_path):
    df_verificacion = pd.read_csv(csv_path)
    print(f"\n📊 Resumen del CSV:")
    print(f"  📈 Total de registros: {len(df_verificacion)}")
    print(f"  📅 Rango de fechas: {df_verificacion['Fecha'].min()} a {df_verificacion['Fecha'].max()}")
    print(f"  🔢 Formato de IDs: {df_verificacion['id_post'].iloc[0]} - {df_verificacion['id_post'].iloc[-1]}")

print("\n🎉 ¡Proceso completado exitosamente!")
print(f"📍 Todos los archivos se encuentran en: {BASE_DIR}")

📁 Directorio del proyecto: /Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DIGITAL_Proyecto Final

📁 Archivos generados:
  ✅ publicaciones_pixabay.csv - 0.03 MB
  ✅ imagenes_pixabay.zip - 14.14 MB

📂 Carpeta de imágenes: /Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DIGITAL_Proyecto Final/imagenes_descargadas
  📸 Imágenes descargadas: 200
  📋 Ejemplos de nombres de archivo: POST_1.jpg, POST_10.jpg, POST_100.jpg, POST_101.jpg, POST_102.jpg, POST_103.jpg, POST_104.jpg, POST_105.jpg, POST_106.jpg, POST_107.jpg

📊 Resumen del CSV:
  📈 Total de registros: 200
  📅 Rango de fechas: 2024-08-05 a 2025-08-01
  🔢 Formato de IDs: POST_98 - POST_153

🎉 ¡Proceso completado exitosamente!
📍 Todos los archivos se encuentran en: /Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DIGITAL_Proyecto Final
